In [1]:
import numpy as np
import os
import random
import scipy.misc
import time

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


In [2]:
class DataHelper:
    def __init__(self, data_file):
        xs = []
        ys = []
        
        with open(data_file) as f:
            header = f.readline()
            dirname = os.path.dirname(data_file)
            for line in f:
                fields = line.split(", ")
                xs.append(os.path.join(dirname, fields[0]))
                ys.append(fields[3])
                
        c = list(zip(xs, ys))
        random.shuffle(c)
        xs, ys = zip(*c)
        
        self._batch_pointer = 0
        self._train_xs = xs[:(int)(len(xs) * 0.9)]
        self._train_ys = ys[:(int)(len(xs) * 0.9)]

        val_xs = []
        val_ys = []
        val_size = (int)(len(xs) * 0.1)
        for i in range(val_size):
            val_ys.append(ys[-i])
            val_xs.append(scipy.misc.imread(xs[-i]))
        
        self._val_xs = np.asarray(val_xs)
        self._val_ys = np.asarray(val_ys)
        
    def data_size(self):
        return len(self._train_ys) + len(self._val_ys)
    
    def val_data(self):
        return self._val_xs, self._val_ys

    def next_train_batch(self, batch_size):
        x_out = []
        y_out = []
        for i in range(batch_size):
            data_idx = (self._batch_pointer + i) % len(self._train_ys)
            y_out.append(self._train_ys[data_idx])
            x_out.append(scipy.misc.imread(self._train_xs[data_idx]))

        self._batch_pointer += batch_size
        return np.asarray(x_out), np.asarray(y_out)

In [3]:
dh = DataHelper('data/driving_log.csv')
print(dh.data_size())

8035


In [4]:
x, y = dh.next_train_batch(64)
print(x.shape, y.shape)

(64, 160, 320, 3) (64,)


In [5]:
def simple_conv():
    model = Sequential()
    
    model.add(Convolution2D(24, 5, 5, input_shape=(160, 320, 3)))
    model.add(Activation('relu'))

    model.add(Convolution2D(36, 5, 5))
    model.add(Activation('relu'))

    model.add(Convolution2D(48, 5, 5))
    model.add(Activation('relu'))

    model.add(Convolution2D(64, 5, 5))
    model.add(Activation('relu'))

    model.add(Convolution2D(64, 5, 5))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(1164))
    model.add(Activation('relu'))
    
    model.add(Dense(100))
    model.add(Activation('relu'))

    model.add(Dense(50))
    model.add(Activation('relu'))

    model.add(Dense(10))
    model.add(Activation('relu'))

    model.add(Dense(1))
    model.add(Activation('tanh'))

    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [6]:
model = simple_conv()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 156, 316, 24)  1824        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 156, 316, 24)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 152, 312, 36)  21636       activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 152, 312, 36)  0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [ ]:
EPOCHS = 1
BATCH_SIZE = 64
DATA_SIZE = dh.data_size()

val_x, val_y = dh.val_data()

print("Starting training")
for steps in range(EPOCHS * DATA_SIZE):
    step_start = time.time()
    cur_epoch = steps / DATA_SIZE
    x, y = dh.next_train_batch(BATCH_SIZE)

    train_loss = model.train_on_batch(x, y)
    val_loss = model.evaluate(val_x, val_y, verbose=0)
    
    time_taken = time.time() - step_start
    print("Steps {} train loss {} validation loss {} time taken {}".format(
            steps, train_loss, val_loss, time_taken))
    
    print("Saving model")
    model.save("model.h5")

Starting training
